# March Madness 2025

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import os
from sklearn.model_selection import train_test_split
import random
from data import Data, STATS_COLUMNS
import model
import elo

torch.manual_seed(20250222)
random.seed(20250222)

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Hypothesis
Each team can be modeled by x hidden features. In each game, these hidden features interact in a nonlinear fashion to determine the outcome of the game

## Preparing the data
Load the data

In [2]:
dataset = Data(batch_size=100)

dataset.games.describe()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT,WFGM,WFGA,WFGM3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
count,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,...,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000,200590.000000
mean,2015.575896,70.371624,2101.677721,74.190697,2099.323680,61.205668,0.061738,26.180557,57.072441,6.919323,...,19.266279,11.437051,16.819468,10.801152,21.937435,11.218625,15.191485,6.454031,2.849489,18.836014
std,6.073611,36.089915,986.770174,11.403172,989.989621,11.373177,0.287218,4.809783,7.820486,3.168482,...,6.327385,5.238806,6.985267,4.417465,4.706791,3.764948,5.029754,2.984269,2.036512,4.586616
min,2003.000000,0.000000,1101.000000,30.000000,1101.000000,11.000000,0.000000,9.000000,26.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.000000
25%,2011.000000,38.000000,1261.000000,66.000000,1254.000000,54.000000,0.000000,23.000000,52.000000,5.000000,...,15.000000,8.000000,12.000000,8.000000,19.000000,9.000000,12.000000,4.000000,1.000000,16.000000
50%,2016.000000,74.000000,1413.000000,74.000000,1407.000000,61.000000,0.000000,26.000000,57.000000,7.000000,...,19.000000,11.000000,16.000000,10.000000,22.000000,11.000000,15.000000,6.000000,3.000000,19.000000
75%,2021.000000,101.000000,3245.000000,81.000000,3246.000000,69.000000,0.000000,29.000000,62.000000,9.000000,...,23.000000,15.000000,21.000000,14.000000,25.000000,14.000000,18.000000,8.000000,4.000000,22.000000
max,2025.000000,132.000000,3480.000000,149.000000,3480.000000,144.000000,6.000000,58.000000,113.000000,30.000000,...,80.000000,48.000000,65.000000,38.000000,53.000000,34.000000,49.000000,26.000000,21.000000,47.000000


The x's will be the indexes of two team IDs and program IDs, the y's will include and indicator of who won at the game stats

## Predicting Stats
The first model we build will predict game statistics. There is more information encapsulated in the box scores than simply wins and losses so we will use this to build up embeddings that are predictive about how the games will play out

In [3]:
stats_train_loader, stats_validation_loader = dataset.train_test_data(output_stats=True, cache="stats", train_size=0.95)

Loading cached data


In [4]:
stats_model = model.StatsModel(program_embedding=512, team_embedding=512,
                               num_programs=len(dataset.programs), num_teams=len(dataset.teams),
                               model_sizes=(512,256), dropout=0.5).to(device)

In [5]:
model.train(stats_train_loader, stats_validation_loader, stats_model, name="stats_model", learning_rate=0.001, use_cache=True)

Epoch   0: Train Loss=312.25820894, Test Loss=313.46606748
Epoch   1: Train Loss=291.93443420, Test Loss=293.27399551
Epoch   2: Train Loss=217.52172819, Test Loss=218.58782216
Epoch   3: Train Loss=139.08427683, Test Loss=139.99511925
Epoch   4: Train Loss=46.93357530, Test Loss=47.33088414
Epoch   5: Train Loss=42.41784430, Test Loss=42.75749985
Epoch   6: Train Loss=42.51587323, Test Loss=42.82346469
Epoch   7: Train Loss=36.94908565, Test Loss=37.34469901
Epoch   8: Train Loss=34.91462834, Test Loss=35.38134381
Epoch   9: Train Loss=34.38478523, Test Loss=34.83901281
Epoch  10: Train Loss=33.87606256, Test Loss=34.45114280
Epoch  11: Train Loss=33.78846729, Test Loss=34.38634837
Epoch  12: Train Loss=33.33935209, Test Loss=33.86367644
Epoch  13: Train Loss=32.71727168, Test Loss=33.29108491
Epoch  14: Train Loss=32.66259234, Test Loss=33.30444612
Epoch  15: Train Loss=31.62069581, Test Loss=32.27759471
Epoch  16: Train Loss=32.44991115, Test Loss=33.10674501
Epoch  17: Train Loss=3

## The Model
Now we use the embeddings developed in the first model to build a model to predict which team will win, which is the thing we are, in the end trying to do.

In [6]:
result_train_loader, result_validation_loader = dataset.train_test_data(output_stats=False, cache="result", train_size=0.95)

Loading cached data


In [7]:
result_model = model.Model(matchup=stats_model.matchup, model_sizes=(512,64), dropout=0.5).to(device)
result_model.matchup.freeze()

## Training the model

Train the model

In [8]:
model.train(result_train_loader, result_validation_loader, result_model, name="result_model", learning_rate=0.0001, use_cache=True)

Epoch   0: Train Loss=0.24812716, Test Loss=0.24810703
Epoch   1: Train Loss=0.16665169, Test Loss=0.16430366
Epoch   2: Train Loss=0.16330363, Test Loss=0.16092641
Epoch   3: Train Loss=0.16123873, Test Loss=0.15875733
Epoch   4: Train Loss=0.16137741, Test Loss=0.15914910
Epoch   5: Train Loss=0.16050475, Test Loss=0.15829218
Epoch   6: Train Loss=0.16102431, Test Loss=0.15899746
Epoch   7: Train Loss=0.16121288, Test Loss=0.15912022
Epoch   8: Train Loss=0.16152199, Test Loss=0.15960058
Epoch   9: Train Loss=0.16137083, Test Loss=0.15940730
Epoch  10: Train Loss=0.16216813, Test Loss=0.16034890
Best Loss: 0.158292


In [9]:
model.print_results(result_validation_loader, result_model, label="Validation Results")

Validation Results: Accuracy=76.42, Loss=0.158292


With this model we can predict the output of about three quarters of regular season games.

## Load the tourney data to test with

In [10]:
tourney_dataset = dataset.gen_dataset(dataset.tourney)
tourney_loader = DataLoader(tourney_dataset, batch_size=500, shuffle=True)

In [11]:
model.print_results(tourney_loader, result_model, label="Tournament Results")

Tournament Results: Accuracy=73.79, Loss=0.168876


When it comes to tournament results we get a little worse. The lower result is likely due to teams having increased pairity.

Train with early torney data

In [12]:
tourney_df = dataset.tourney[dataset.tourney.Season < 2021]

tourney_train_df, tourney_validation_df = train_test_split(tourney_df, train_size=0.8)
tourney_train_data = dataset.gen_dataset(tourney_train_df)
tourney_validation_data = dataset.gen_dataset(tourney_validation_df)

tourney_train_loader = DataLoader(tourney_train_data, batch_size=500)
tourney_validation_loader = DataLoader(tourney_validation_data, batch_size=500)

In [13]:
result_model.matchup.freeze()

In [14]:
model.train(tourney_train_loader, tourney_validation_loader, result_model, name="tuned_model", learning_rate=0.00001)

Epoch   0: Train Loss=0.16754529, Test Loss=0.16192086
Epoch   1: Train Loss=0.16752098, Test Loss=0.16192846
Epoch   2: Train Loss=0.16751234, Test Loss=0.16194719
Epoch   3: Train Loss=0.16751614, Test Loss=0.16197599
Epoch   4: Train Loss=0.16752720, Test Loss=0.16201375
Epoch   5: Train Loss=0.16754147, Test Loss=0.16205508
Best Loss: 0.161921


### Performance by year


In [15]:
for season in dataset.tourney.Season.unique():
    loader = dataset.tourney_data(year=season)
    model.print_results(loader, result_model, label=f"{season} Tournament")

2003 Tournament: Accuracy=75.00, Loss=0.173870
2004 Tournament: Accuracy=68.75, Loss=0.188508
2005 Tournament: Accuracy=75.78, Loss=0.172403
2006 Tournament: Accuracy=71.09, Loss=0.205128
2007 Tournament: Accuracy=75.00, Loss=0.173933
2008 Tournament: Accuracy=76.56, Loss=0.160642
2009 Tournament: Accuracy=78.12, Loss=0.159300
2010 Tournament: Accuracy=74.02, Loss=0.161072
2011 Tournament: Accuracy=72.31, Loss=0.169797
2012 Tournament: Accuracy=75.38, Loss=0.158520
2013 Tournament: Accuracy=71.15, Loss=0.179299
2014 Tournament: Accuracy=74.62, Loss=0.165283
2015 Tournament: Accuracy=78.46, Loss=0.145624
2016 Tournament: Accuracy=70.38, Loss=0.172568
2017 Tournament: Accuracy=77.69, Loss=0.151417
2018 Tournament: Accuracy=70.77, Loss=0.177311
2019 Tournament: Accuracy=75.38, Loss=0.149301
2021 Tournament: Accuracy=70.54, Loss=0.180634
2022 Tournament: Accuracy=72.76, Loss=0.180686
2023 Tournament: Accuracy=73.51, Loss=0.183704
2024 Tournament: Accuracy=72.01, Loss=0.159995


In [16]:
stage1_loader = dataset.tourney_data(after=2021)
model.print_results(stage1_loader, result_model, label=f"Stage 1")

Stage 1: Accuracy=72.22, Loss=0.176213


Breaking out by league

In [17]:
for season in dataset.tourney.Season.unique():
    for league in dataset.tourney[dataset.tourney.Season == season].League.unique():
        loader = dataset.tourney_data(year=season, league=league)
        model.print_results(loader, result_model, label=f"{season} {league} Tournament")

2003 M Tournament: Accuracy=75.00, Loss=0.173870
2004 M Tournament: Accuracy=68.75, Loss=0.188508
2005 M Tournament: Accuracy=75.78, Loss=0.172403
2006 M Tournament: Accuracy=71.09, Loss=0.205128
2007 M Tournament: Accuracy=75.00, Loss=0.173933
2008 M Tournament: Accuracy=76.56, Loss=0.160642
2009 M Tournament: Accuracy=78.12, Loss=0.159300
2010 M Tournament: Accuracy=70.31, Loss=0.186961
2010 W Tournament: Accuracy=77.78, Loss=0.134772
2011 M Tournament: Accuracy=65.67, Loss=0.213896
2011 W Tournament: Accuracy=79.37, Loss=0.122898
2012 M Tournament: Accuracy=69.40, Loss=0.189886
2012 W Tournament: Accuracy=81.75, Loss=0.125161
2013 M Tournament: Accuracy=67.16, Loss=0.202641
2013 W Tournament: Accuracy=75.40, Loss=0.154474
2014 M Tournament: Accuracy=67.16, Loss=0.203422
2014 W Tournament: Accuracy=82.54, Loss=0.124722
2015 M Tournament: Accuracy=74.63, Loss=0.165671
2015 W Tournament: Accuracy=82.54, Loss=0.124304
2016 M Tournament: Accuracy=65.67, Loss=0.192182
2016 W Tournament: A

## Inspect the model
First what are the sizes of the smallest input and output weights

In [18]:
print(f"Program embedding min: {result_model.matchup.program_embedding.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")
print(f"Team embedding min: {result_model.matchup.team_embedding.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")
print(f"FC min: {result_model.fc3.state_dict()['weight'].abs().max(axis=0).values.min().item():>8f}")

Program embedding min: 2.394429
Team embedding min: 3.582119
FC min: 0.005568


Calculate the average gradient for each input feature

In [19]:
result_model.matchup.unfreeze()

In [20]:
program_weights, team_weights, stats_weights = model.feature_eval(result_model, tourney_loader)

In [21]:
program_weights.abs().sum().item(), team_weights.abs().sum().item()

(0.00032028782879933715, 0.00028570747235789895)

In [22]:
print(f"Year:\t{stats_weights[0]:>4f}")
print(f"Game:\t{stats_weights[1]:>4f}")
print(f"League:\t{stats_weights[2]:>4f}")

Year:	0.000104
Game:	-0.000674
League:	0.002089


## Save the model

In [23]:
torch.save(result_model.state_dict(), 'model.pth')

## Moderated model

Moderate a model by pushing it towards 0.5

In [24]:
moderated = model.ModeratedModel(result_model, 0.75)
model.print_results(stage1_loader, moderated, label="Moderated Model")

Moderated Model: Accuracy=72.22, Loss=0.179975


In [25]:
for season in dataset.tourney.Season.unique():
    loader = dataset.tourney_data(season)
    model.print_results(loader, moderated, label=f"{season} Tournament")

2003 Tournament: Accuracy=75.00, Loss=0.178965
2004 Tournament: Accuracy=68.75, Loss=0.191104
2005 Tournament: Accuracy=75.78, Loss=0.177116
2006 Tournament: Accuracy=71.09, Loss=0.203641
2007 Tournament: Accuracy=75.00, Loss=0.180710
2008 Tournament: Accuracy=76.56, Loss=0.170574
2009 Tournament: Accuracy=78.12, Loss=0.169275
2010 Tournament: Accuracy=74.02, Loss=0.168765
2011 Tournament: Accuracy=72.31, Loss=0.174562
2012 Tournament: Accuracy=75.38, Loss=0.167384
2013 Tournament: Accuracy=71.15, Loss=0.182222
2014 Tournament: Accuracy=74.62, Loss=0.171779
2015 Tournament: Accuracy=78.46, Loss=0.156324
2016 Tournament: Accuracy=70.38, Loss=0.176118
2017 Tournament: Accuracy=77.69, Loss=0.160126
2018 Tournament: Accuracy=70.77, Loss=0.179441
2019 Tournament: Accuracy=75.38, Loss=0.158663
2021 Tournament: Accuracy=70.54, Loss=0.182246
2022 Tournament: Accuracy=72.76, Loss=0.183749
2023 Tournament: Accuracy=73.51, Loss=0.186682
2024 Tournament: Accuracy=72.01, Loss=0.167306


It underperforms even in years where there were upsets.

## Dig into 2023 results

In [26]:
loader = dataset.tourney_data(2023)

x, y = loader.dataset.tensors

preds = result_model(x.to(device))

In [27]:
t_2023 = pd.DataFrame({'winner_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,0].tolist()],
                       'loser_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,2].tolist()],
                       'winner': [dataset.programs.loc[i].TeamID for i in x[:,0].tolist()],
                       'loser': [dataset.programs.loc[i].TeamID for i in x[:,2].tolist()],
                       'actual': y.reshape(-1),
                       'predicted': preds.cpu().detach().numpy().reshape(-1)}).iloc[:67]

In [28]:
t_2023[t_2023.predicted < 0.5].sort_values('predicted')

,winner_name,loser_name,winner,loser,actual,predicted
23,F Dickinson,Purdue,1192,1345,1.0,0.027766
15,Princeton,Arizona,1343,1112,1.0,0.112973
8,Furman,Virginia,1202,1438,1.0,0.193283
39,Princeton,Missouri,1343,1281,1.0,0.249799
53,FL Atlantic,Tennessee,1194,1397,1.0,0.269607
58,San Diego St,Alabama,1361,1104,1.0,0.308628
14,Penn St,Texas A&M,1336,1401,1.0,0.320116
57,Miami FL,Houston,1274,1222,1.0,0.333005
50,Michigan St,Marquette,1277,1266,1.0,0.374013
54,Gonzaga,UCLA,1211,1417,1.0,0.381236


The biggest thing in this season were the huge upsets in the first round. Purdue was a number one seed and lost which I only gave a 1% chance to happen. Arizona and Virginia were number 2 seeds and lost which I gave 13% and 17% chances of happening respectively.

In [29]:
t_2023['Upset'] = [dataset.upset(2023, winner, loser) for (winner, loser) in zip(t_2023['winner'], t_2023['loser'])]

In [30]:
t_2023[t_2023.Upset].predicted.mean()

np.float64(0.3622406880456919)

On average the upsets had a 36% chance of happening

In [31]:
t_2023[t_2023.Upset & (t_2023.predicted >= 0.5)].sort_values('predicted', ascending=False)

,winner_name,loser_name,winner,loser,actual,predicted,Upset
60,Connecticut,Gonzaga,1163,1211,1.0,0.531841,True
45,Creighton,Baylor,1166,1124,1.0,0.530038,True
6,Auburn,Iowa,1120,1234,1.0,0.524678,True


I correctly predicted 4 upsets, though all were closely ranked

In [32]:
t_2023[~t_2023.Upset & (t_2023.predicted < 0.5)].sort_values('predicted')

,winner_name,loser_name,winner,loser,actual,predicted,Upset
11,Maryland,West Virginia,1268,1452,1.0,0.448298,False
28,Kentucky,Providence,1246,1344,1.0,0.461974,False
63,San Diego St,Creighton,1361,1166,1.0,0.469778,False
2,Arizona St,Nevada,1113,1305,1.0,0.481851,False
48,Kansas St,Kentucky,1243,1246,1.0,0.488031,False
12,Missouri,Utah St,1281,1429,1.0,0.494350,False
5,Arkansas,Illinois,1116,1228,1.0,0.495114,False


I also incorrectly predicted 3 upsets

Looking at all the tourneys

In [33]:
x, y = tourney_loader.dataset.tensors
preds = result_model(x.to(device))
tourney_df = pd.DataFrame({'season': x[:,4].tolist(),
                           'winner_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,0].tolist()],
                           'loser_name': [dataset.all_teams.loc[dataset.programs.loc[i].TeamID].TeamName for i in x[:,2].tolist()],
                           'winner': [dataset.programs.loc[i].TeamID for i in x[:,0].tolist()],
                           'loser': [dataset.programs.loc[i].TeamID for i in x[:,2].tolist()],
                           'actual': y.reshape([-1]),
                           'predicted': np.array(preds.tolist()).reshape([-1])})
tourney_df = tourney_df[tourney_df.actual == 1.0]
tourney_df['Upset'] = [dataset.upset(season, winner, loser) for (winner, loser, season)
                       in zip(tourney_df['winner'], tourney_df['loser'], tourney_df['season'])]

In [34]:
len(tourney_df[tourney_df.Upset & (tourney_df.predicted >= 0.5) & (tourney_df.season > 2020)].sort_values('predicted', ascending=False))

18

In [35]:
len(tourney_df[~tourney_df.Upset & (tourney_df.predicted < 0.5) & (tourney_df.season > 2020)].sort_values('predicted'))

44

Overall I predicted 30 upsets correctly, and 35 incorrectly

## Predicting by seeds
What if I predict just using the seeds?

In [36]:
odds = dataset.odds_by_seed_diff(before=2021)

In [37]:
dataset.tourney_df(after=2021).SeedDiff.map(lambda x: odds[x]**2).mean()

np.float64(0.18527460145235355)

This results in a test Brier score of about 0.185.

## Using Elo
I can also compute Elo rankings

In [38]:
elo_model = elo.EloModel(dataset, k=50)
model.print_results(stage1_loader, elo_model, label="Elo Model")

Elo Model: Accuracy=73.45, Loss=0.178886


It is similar to our model

## Hybrid Model
Building a model using the neural net and seeds

In [39]:
seed_model = model.SeedModel(dataset)
model.print_results(stage1_loader, seed_model, label="Seed Model")
model.print_results(stage1_loader, elo_model, label="Elo Model")
model.print_results(stage1_loader, result_model, label="Base NN")

Seed Model: Accuracy=72.03, Loss=0.184022
Elo Model: Accuracy=73.45, Loss=0.178886
Base NN: Accuracy=72.22, Loss=0.176213


In [40]:
hybrid = model.HybridModel([result_model, seed_model, elo_model], [0.7, 0.2, 0.1])

model.print_results(stage1_loader, hybrid, label="Hybrid Model")

Hybrid Model: Accuracy=72.32, Loss=0.175277


They hybrid model outperforms all individual models

In [41]:
for season in range(2021, 2025):
    for league in ['M', 'W']:
        loader = dataset.tourney_data(year=season, league=league)
        model.print_results(loader, hybrid, label=f"{season} {league} Tournament")

2021 M Tournament: Accuracy=62.12, Loss=0.215867
2021 W Tournament: Accuracy=78.57, Loss=0.145708
2022 M Tournament: Accuracy=69.40, Loss=0.209067
2022 W Tournament: Accuracy=75.37, Loss=0.149548
2023 M Tournament: Accuracy=71.64, Loss=0.202040
2023 W Tournament: Accuracy=77.61, Loss=0.163325
2024 M Tournament: Accuracy=66.42, Loss=0.185721
2024 W Tournament: Accuracy=77.61, Loss=0.129777


## Generating the submission file
### Phase 2

Write the results

In [42]:
model.gen_submission(hybrid, dataset, fname="hybrid.csv")

In [43]:
model.gen_submission(result_model, dataset, fname="base.csv")

## Generate a bracket

With the hybrid model

In [44]:
season, league = 2025, 'M'

In [45]:
bracket = model.gen_bracket(dataset, season, league, hybrid).join(dataset.all_teams, on='Winner')[['Winner', 'TeamName']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(bracket)

      Winner        TeamName
Slot                        
R1W1    1181            Duke
R1W2    1104         Alabama
R1W3    1458       Wisconsin
R1W4    1112         Arizona
R1W5    1332          Oregon
R1W6    1140             BYU
R1W7    1388    St Mary's CA
R1W8    1124          Baylor
R1X1    1222         Houston
R1X2    1397       Tennessee
R1X3    1246        Kentucky
R1X4    1345          Purdue
R1X5    1155         Clemson
R1X6    1228        Illinois
R1X7    1417            UCLA
R1X8    1211         Gonzaga
R1Y1    1120          Auburn
R1Y2    1277     Michigan St
R1Y3    1235         Iowa St
R1Y4    1401       Texas A&M
R1Y5    1276        Michigan
R1Y6    1314  North Carolina
R1Y7    1266       Marquette
R1Y8    1257      Louisville
R1Z1    1196         Florida
R1Z2    1385       St John's
R1Z3    1403      Texas Tech
R1Z4    1268        Maryland
R1Z5    1272         Memphis
R1Z6    1281        Missouri
R1Z7    1242          Kansas
R1Z8    1163     Connecticut
R2W1    1181  

In [46]:
model.tiebreaker(bracket, stats_model, dataset, season, league)

139.47025781813414

And with the base model

In [47]:
base_bracket = model.gen_bracket(dataset, 2025, 'M', result_model).join(dataset.all_teams, on='Winner')[['Winner', 'TeamName']]
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(base_bracket)

      Winner        TeamName
Slot                        
R1W1    1181            Duke
R1W2    1104         Alabama
R1W3    1458       Wisconsin
R1W4    1112         Arizona
R1W5    1332          Oregon
R1W6    1140             BYU
R1W7    1388    St Mary's CA
R1W8    1124          Baylor
R1X1    1222         Houston
R1X2    1397       Tennessee
R1X3    1246        Kentucky
R1X4    1345          Purdue
R1X5    1155         Clemson
R1X6    1228        Illinois
R1X7    1417            UCLA
R1X8    1211         Gonzaga
R1Y1    1120          Auburn
R1Y2    1277     Michigan St
R1Y3    1235         Iowa St
R1Y4    1401       Texas A&M
R1Y5    1276        Michigan
R1Y6    1314  North Carolina
R1Y7    1266       Marquette
R1Y8    1257      Louisville
R1Z1    1196         Florida
R1Z2    1385       St John's
R1Z3    1403      Texas Tech
R1Z4    1268        Maryland
R1Z5    1272         Memphis
R1Z6    1281        Missouri
R1Z7    1242          Kansas
R1Z8    1163     Connecticut
R2W1    1181  

In [48]:
model.tiebreaker(base_bracket, stats_model, dataset, season, league)

139.47025781813414